# Notebook for some visualization

In [2]:
import sys
sys.path.append('..')

### Visualizing the surface code

In [3]:
import stim 
import numpy as np
from config import Config 

config = Config()

d = config.code_distance
code_task = config.code_task
rounds = config.rounds

p = 0.1 # set manually

circuit = stim.Circuit.generated(
    code_task,
        distance=d,
        rounds=rounds,
        after_clifford_depolarization=p,          
        after_reset_flip_probability=2*p,        
        before_measure_flip_probability=0.0,      # Set to 0.0 (handled by Soft Model)
        before_round_data_depolarization=p/10,    # Idle noise on data qubits between rounds
)

In [4]:
flat = circuit.flattened()

for instruction in flat:
    if instruction.name == "QUBIT_COORDS":
        print(instruction)

QUBIT_COORDS(1, 1) 1
QUBIT_COORDS(2, 0) 2
QUBIT_COORDS(3, 1) 3
QUBIT_COORDS(5, 1) 5
QUBIT_COORDS(1, 3) 8
QUBIT_COORDS(2, 2) 9
QUBIT_COORDS(3, 3) 10
QUBIT_COORDS(4, 2) 11
QUBIT_COORDS(5, 3) 12
QUBIT_COORDS(6, 2) 13
QUBIT_COORDS(0, 4) 14
QUBIT_COORDS(1, 5) 15
QUBIT_COORDS(2, 4) 16
QUBIT_COORDS(3, 5) 17
QUBIT_COORDS(4, 4) 18
QUBIT_COORDS(5, 5) 19
QUBIT_COORDS(4, 6) 25


In [5]:
# This function only works for this specific type of surface code, rotated_memory_z, and currently only d = 3
# Maybe in the future could think about trying to create a function that can do this for any general circuit

def calculate_final_round_stabilizers(data_qubits, d = config.code_distance):
    # I have just done this manually which isn't very good, so need to think how to generalize this

    # Manually create code structure
    # Dictionary for Z-measurements, note we leave X measurements as a default -1 (chosen embedding for null measurement)
    # Keys = stabilizer number (1-indexed), Values = data qubit indices (0-indexed) that participate

    code_z_structure = {1: [0, 1], 3: [1, 2, 4, 5], 6: [3, 4, 6, 7], 8: [7, 8]}
    stabilizers = d*d - 1   # Number of stabilizers

    num_shots = data_qubits.shape[0]
    stabilizer_values = np.full((num_shots, stabilizers), -1, dtype=data_qubits.dtype)   # Default -1 for X measurements

    for z, idxs in code_z_structure.items():
        stabilizer_values[:, z-1] = np.sum(data_qubits[:, idxs], axis=1) % 2   # Parity of data qubit measurements

    print(stabilizer_values)
    
    return stabilizer_values


In [6]:
from data_utils.noise_model import SoftNoiseModel

# Note: All documentation for stim can be found on github in doc -> python_api_reference_vDev.md 

def generate_SI1000_training_data(p: np.float32, d=config.code_distance, rounds=config.rounds, shots=config.shots, code_task = config.code_task):
    """Generate training data with soft information."""
    
    # 1. Create Stim circuit
    # Note: We set measurement error to 0 here because we will simulate 
    # the realistic measurement noise manually using the SoftNoiseModel.

    # stim.Circuit.generated generates common circuits such as the rotated_memory_z code we are using
    circuit = stim.Circuit.generated(
        code_task,
        distance=d,
        rounds=rounds,
        after_clifford_depolarization=p,          
        after_reset_flip_probability=2*p,        
        before_measure_flip_probability=0.0,      # Set to 0.0 (handled by Soft Model)
        before_round_data_depolarization=p/10,    # Idle noise on data qubits between rounds
    )

    # We generate a random seed explicitly so we can pass it to both samplers.
    # This ensures the "Raw Measurements" and "Logical Errors" come from the 
    # exact same simulation trajectory.

    # Note: We need to do this as compile_detector_sampler.sample() returns syndromes (parity checks) not the raw data we want for our model
    current_seed = np.random.randint(0, 2**31 - 1)

    # 2. Get Ground Truth / Raw Data
    # Pass the seed here
    # 2. Get Ground Truth / Raw Data
    sampler = circuit.compile_sampler(seed=current_seed)
    raw_measurements = sampler.sample(shots=shots).astype(np.float32)

    # Get logical labels (same seed for identical trajectory)
    detector_sampler = circuit.compile_detector_sampler(seed=current_seed)
    _, logical_errors = detector_sampler.sample(shots=shots, separate_observables=True)
    
    # 3. Apply Soft Noise Model 
    noise_model = SoftNoiseModel(snr=10.0, t_integration=0.01)
    z_values = noise_model.generate_signal(raw_measurements)
    post_1, post_2 = noise_model.calculate_posteriors(z_values)
    
    # Calculate shapes dynamically to be safe
    # For rotated memory z, measurements = rounds * (d^2-1) + d^2
    num_stabilizers = d**2 - 1 
    
    end_of_bulk = rounds * num_stabilizers

    # Expected total number of measurements in raw_measurements (per shot)
    expected_total = end_of_bulk + (d*d)
    
    # Safety Check: Ensure the circuit actually produced the expected number of bits
    assert raw_measurements.shape[1] == expected_total, \
        f"Mismatch in expected circuit output. Expected {end_of_bulk + d*d}, got {raw_measurements.shape[1]}"

    bulk_post_1 = post_1[:, :end_of_bulk].reshape(shots, rounds, num_stabilizers)
    bulk_post_2 = post_2[:, :end_of_bulk].reshape(shots, rounds, num_stabilizers)

    prev_post_1 = np.zeros_like(bulk_post_1)
    prev_post_1[:, 1:, :] = bulk_post_1[:, :-1, :] 
    
    prev_post_2 = np.zeros_like(bulk_post_2)
    prev_post_2[:, 1:, :] = bulk_post_2[:, :-1, :] 

    events_1 = noise_model.soft_xor(bulk_post_1, prev_post_1)
    events_2 = noise_model.soft_xor(bulk_post_2, prev_post_2)

    events_1 = events_1.reshape(shots, -1)
    events_2 = events_2.reshape(shots, -1)

    # Threshold so can't learn inverse of SoftXOR
    num_final_qubits = d * d
    post_1[:, -num_final_qubits:] = (post_1[:, -num_final_qubits:] > 0.5).astype(np.float32)
    post_2[:, -num_final_qubits:] = (post_2[:, -num_final_qubits:] > 0.5).astype(np.float32)

    # Now convert the data qubits into stabilizer measurements
    final_stabilizers_1 = calculate_final_round_stabilizers(post_1[:, -num_final_qubits:], d)
    final_stabilizers_2 = np.zeros((shots, num_final_qubits-1))

    # Replace the last num_final_qubits columns with the num_final_qubits - 1 stabilizer values
    post_1 = np.concatenate([post_1[:, :-num_final_qubits], final_stabilizers_1], axis=1).astype(np.float32)
    post_2 = np.concatenate([post_2[:, :-num_final_qubits], final_stabilizers_2], axis=1).astype(np.float32)

    return post_1, events_1, post_2, events_2, logical_errors

In [7]:
post_1, events_1, post_2, events_2, logical_errors = generate_SI1000_training_data(0.1, shots=1)

[[ 1. -1.  0. -1. -1.  0. -1.  1.]]


In [8]:
print(post_1)
print(post_2)
print(events_1)
print(events_2)

[[ 0.00553839  0.00382614  0.00163055  0.9999995   0.00421505  0.00327277
   0.00190341  0.99999404  0.9976447   0.99999255  0.0058239   0.9999769
   0.99999857  0.00509036  0.99968755  0.00893149  0.9902456   0.00617062
   0.00495758  0.7331416   0.01754442  1.          0.9994979   0.99313456
   1.         -1.          0.         -1.         -1.          0.
  -1.          1.        ]]
[[4.09051104e-09 1.03194708e-09 5.69126170e-12 1.13238435e-04
  1.54180213e-09 5.02861586e-10 1.88578458e-11 4.83875592e-05
  1.72010605e-05 4.58456525e-05 4.78950835e-09 3.56881392e-05
  7.32799381e-05 3.09472914e-09 2.27164819e-05 1.49320165e-08
  1.44183978e-05 5.70114178e-09 2.82467982e-09 7.32836770e-06
  5.61609177e-08 9.72854323e-04 2.11884053e-05 1.50547748e-05
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
[[5.53839091e-03 3.82614043e-03 1.63055144e-03 9.99999534e-01
  4.21505064e-03 3.27277259e-03 1.90341442e-03 9.99